In [7]:
!pip install tensorflow --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.11.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached h5py-3.14.0-cp312-cp312-win_amd64.whl.metadata (2.7 kB)
  Using cached m

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [5]:
# Import necessary libraries for neural network implementation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load the fuel consumption dataset
df = pd.read_csv('Fuel_Consumption_2000-2022.csv')
df.head()

In [ ]:
# Display basic information about the dataset
df.info()

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Display statistical summary of numerical columns
df.describe()

In [ ]:
# Visualize correlation matrix of numerical features
numeric_cols = df.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(12, 8))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Numerical Features')
plt.show()

In [ ]:
# Create fuel efficiency categories for classification (low, medium, high efficiency)
df['FUEL_EFFICIENCY'] = pd.cut(df['FUEL CONSUMPTION'], 
                              bins=[0, 8, 12, float('inf')], 
                              labels=['High', 'Medium', 'Low'])
df['FUEL_EFFICIENCY'].value_counts()

In [ ]:
# Prepare independent variables (features)
feature_columns = ['YEAR', 'ENGINE SIZE', 'CYLINDERS', 'HWY (L/100 km)', 'COMB (L/100 km)', 'EMISSIONS']
X = df[feature_columns].copy()
X.head()

In [ ]:
# Prepare dependent variable (target)
y = df['FUEL_EFFICIENCY'].copy()
y.head()

In [ ]:
# Encode categorical variables using LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_encoded[:10]

In [ ]:
# Split dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)
print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

In [ ]:
# Perform feature scaling using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f"Scaled training features shape: {X_train_scaled.shape}")
print(f"Scaled testing features shape: {X_test_scaled.shape}")

In [ ]:
# Initialize Artificial Neural Network with 2 hidden layers of 6 neurons each
model = Sequential()

# First hidden layer with 6 neurons and ReLU activation
model.add(Dense(6, input_dim=X_train_scaled.shape[1], activation='relu'))

# Second hidden layer with 6 neurons and tanh activation
model.add(Dense(6, activation='tanh'))

# Output layer with 3 neurons for 3 classes and softmax activation
model.add(Dense(3, activation='softmax'))

model.summary()

In [ ]:
# Compile the neural network with Adam optimizer, categorical crossentropy loss and accuracy metric
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# Fit the neural network with batch_size=32 and epochs=100
history = model.fit(X_train_scaled, y_train, 
                    batch_size=32, 
                    epochs=100, 
                    validation_data=(X_test_scaled, y_test), 
                    verbose=0)

print("Neural network training completed!")

In [ ]:
# Evaluate the model performance on test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test_scaled, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_classes, target_names=le.classes_))

In [ ]:
# Plot training history (accuracy and loss)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

# Plot loss
ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Alternative model with different activation functions (ReLU, Sigmoid)
model2 = Sequential()

# First hidden layer with 6 neurons and ReLU activation
model2.add(Dense(6, input_dim=X_train_scaled.shape[1], activation='relu'))

# Second hidden layer with 6 neurons and sigmoid activation
model2.add(Dense(6, activation='sigmoid'))

# Output layer with 3 neurons and softmax activation
model2.add(Dense(3, activation='softmax'))

# Compile and train the alternative model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history2 = model2.fit(X_train_scaled, y_train, batch_size=32, epochs=100, 
                      validation_data=(X_test_scaled, y_test), verbose=0)

# Evaluate alternative model
test_loss2, test_accuracy2 = model2.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Alternative Model Test Accuracy: {test_accuracy2:.4f}")